# Cleaning for User Timeline Data

- Step0: import modules, read files
- Step1: get page view, and delete unuseful columns
- Step2: match log event names
- Step3: process timestamp
- Step4: Refine some logs
- Step5: get session number
- Group Them All and Export


## Step0: Import Modules

In [1]:
import pandas as pd
import os
import numpy as np

## Step1: get page view

**Find the View depending on log file structure**
1. clean_ref(col_name, df) for those with 'referer' column
2. assign_view(row) for those without 'referer' column but has 'uuid' (identifier only)

In [2]:
def clean_ref(col_name, df):
    # Locate the visualization views
    df['view'] = df[col_name].apply(lambda x: 'Node-Link' if 'nodelink' in str(x).lower() else
                                           'Matrix' if 'matrix' in str(x).lower() else
                                           'Timeline' if 'dynamicego' in str(x).lower() else
                                           'Coordinated' if 'mat-nl' in str(x).lower() else
                                           'Map' if 'map' in str(x).lower() else x)
    # Clean the rest and delete unuseful columns
    df.loc[df['view'].str.contains('vistorian', case=False), 'view'] = ''
    df.drop(columns=[col_name], inplace=True)
    return df

In [3]:
def assign_view(row):
    for col in ['action', 'label', 'value']:
        cell_value = str(row[col])
        if 'nodelink' in cell_value:
            return 'Node-Link'
        elif 'map' in cell_value:
            return 'Map'
        elif 'matrix' in cell_value:
            return 'Matrix'
        elif 'mat-nl' in cell_value:
            return 'Coordinated'
        elif 'dynamicego' in cell_value:
            return 'Timeline'
    return ''

## Step2: Match log_ID with titles and category

In [4]:
def join_log_info(df):
    # Perform inner join
    merged_df = pd.merge(df, df_log, left_on='category', right_on='log_ID')
    
    # Drop the redundant 'log_ID' column
    merged_df = merged_df.drop(columns=['log_ID'])
    
    # Sort the merged DataFrame by 'ts' in ascending order
    merged_df = merged_df.sort_values(by='ts', ascending=True)
    merged_df = merged_df.rename(columns={'category': 'logID'})
    
    return merged_df

In [5]:
def update_log_info(df):
    # Append '_click' to the 'log_title' column where 'value' is equal to 'Click', apply to Node and Link logs
    df.loc[df['value'] == 'Click', 'log_title'] = df.loc[df['value'] == 'Click', 'log_title'] + '_click'
    
    # Append labelling type to 'vis_9' 0=automatic/importance, 1=hide all, 2=show all
    df.loc[(df['value'] == "0") & (df['logID'] == 'vis_9'), 'log_title'] = df.loc[(df['value'] == "0") & (df['logID'] == 'vis_9'), 'log_title'] + '-Automatic/importance'
    df.loc[(df['value'] == "1") & (df['logID'] == 'vis_9'), 'log_title'] = df.loc[(df['value'] == "1") & (df['logID'] == 'vis_9'), 'log_title'] + '-Hide All'
    df.loc[(df['value'] == "2") & (df['logID'] == 'vis_9'), 'log_title'] = df.loc[(df['value'] == "2") & (df['logID'] == 'vis_9'), 'log_title'] + '-Show All'
    
    # Update "Column Specified"
    if len(df.loc[df['logID'] == 'dat_11']) > 0:
        value_to_append = df.loc[df['logID'] == 'dat_11', 'value'].astype(str).values[0]
        df.loc[df['logID'] == 'dat_11', 'log_title'] = df.loc[df['logID'] == 'dat_11', 'log_title'].astype(str) + value_to_append

    # Drop column 'value'
    df.drop(columns=['value'], inplace=True)
    
    return df

In [6]:
def get_category_index(df, file_name=None):
    log_category_index_list = [
        { "category": "data", "index": 0 },
        { "category": "visualization", "index": 1 },
        { "category": "logging", "index": 2 },
        { "category": "help", "index": 3 },
        { "category": "communication", "index": 4 },
        { "category": "error", "index": 5 },
        { "category": "bookmarking", "index": 6 },
    ]
    
    category_to_index = {item["category"]: item["index"] for item in log_category_index_list}
    df['log_category_index'] = df['log_category'].map(category_to_index)
    
    # Find and print the file name and index for the NaN values
    if file_name is not None:
        nan_indices = df[df['log_category_index'].isna()].index
        for index in nan_indices:
            print(f"File: {file_name}, Index: {index}")
    
    # Fill NaN values with a default value (e.g., -1) before converting to integer data type
    df['log_category_index'] = df['log_category_index'].fillna(-1).astype(int)
    
    return df

In [7]:
def decluster_rows(df):
    removed_logs = ['_trace', 'log', 'log_1', 'log_3', 'log_11', 'log_5', 'log_6', 'log_7', 'log_9','log_10', 'log_11', 'log_13', 'log_18', 'log_14', 'log_16', 'log_17', 'log_18','dat_9', 'dat_12','dat_17', 'dat_18', 'dat_19']
    df = df[~df['logID'].isin(removed_logs)]
    
    # Usually the first log is Visit Page, we should remove that
    df = df.iloc[1:]
    
    return df

## Step3: Sort, Combine by Timestamp

In [8]:
def relative_time(df):
    # Add a new index column to the DataFrame
    df = df.reset_index(drop=True).reset_index()
    
    # Calculate the time relative to the first timestamp
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    df['relative_time'] = df['ts'].apply(lambda x: (x - df['ts'].iloc[0]).total_seconds())
    
    # Drop the original ts column
    df.drop(columns=['ts'], inplace = True)
    
    return df

In [9]:
def combine_logs(df):
    # Get a new column for start time
    df = df.assign(start=df['relative_time'])
    
    # Create an empty DataFrame to store the combined rows
    combined_data = pd.DataFrame(columns=["index", "user", "logID", "view", "log_title",
                                        "log_category","log_category_index", "relative_time", "start", "end"])
    
    # Iterate through the rows of the original DataFrame
    i = 0
    while i < len(df):
        row = df.iloc[i]
        index = row["index"]
        user = row["user"]
        logID = row["logID"]
        view = row["view"]
        log_title = row["log_title"]
        log_category = row["log_category"]
        log_category_index = row["log_category_index"]
        relative_time = row["relative_time"]
        start = row['start']
        end = start + 1
#         start = round(row["start"], ndigits=0)
#         end = round(start + 1, ndigits=0)

        # Check if the current row can be combined with the next row(s)
        j = i + 1
        while j < len(df):
            next_row = df.iloc[j]
            if next_row["log_title"] == log_title and next_row["start"] - end < 1:
                end = next_row["start"] + 1
#                 end = round(end, ndigits=0)
                j += 1
            else:
                break

        # Add the combined row to the new DataFrame
        new_row = pd.DataFrame({"index": [index],
                                "user": [user],
                                "logID": [logID],
                                "view": [view],
                                "log_title": [log_title],
                                "log_category": [log_category],
                                "log_category_index": [log_category_index],
                                "relative_time": [relative_time],
                                "start": [start],
                                "end": [end]})
        combined_data = pd.concat([combined_data, new_row], ignore_index=True)

        # Update the index to continue from the last checked row
        i = j
    
    # Round the start and end to integer
    combined_data['start'] = combined_data['start'].astype(int)
    combined_data['end'] = combined_data['end'].astype(int)
    

    # Drop the original ts column
    combined_data.drop(columns=['relative_time', 'index'], inplace=True)
    
    return combined_data

## Step4: Refine some logs

In [10]:
def refine_logs(df):
    # Update Visiting any Vistorian Page to 'Visit Page X'
    df['log_title'] = df.apply(lambda row: 'Visit Page ' + row['view'] if row['log_title'] == 'Visiting any Visitorian Page' else row['log_title'], axis=1)
    return df

In [11]:
def fill_vis_view(df):
    df = df.fillna('')
    
    # Replace empty cells with NaN in the 'view' column
    df.loc[(df['log_category'] == 'visualization') & (df['view'] == ''), 'view'] = np.nan
    
    # Forward fill NaN values in the entire 'view' column
    df['view'] = df['view'].ffill()
    
    return df

## Step5: Is she/he a Multiple-Session user?
**Break sessions by 20min inactivity in logs**

In [12]:
def assign_session(df):
    df['userSession'] = 1
    previous_session_end = df.iloc[0]['end']

    for index, row in df[1:].iterrows():
        if row['start'] - previous_session_end >= 1200:
            df.at[index, 'userSession'] = df.at[index - 1, 'userSession'] + 1
        else:
            df.at[index, 'userSession'] = df.at[index - 1, 'userSession']

        previous_session_end = row['end']
    return df

In [13]:
def reset_time(df):
    unique_sessions = df['userSession'].unique()

    for session_num in unique_sessions:
        if session_num > 1:
            session_start = df[df['userSession'] == session_num].iloc[0]['start']

            df.loc[df['userSession'] == session_num, 'start'] -= session_start
            df.loc[df['userSession'] == session_num, 'end'] -= session_start
    return df

## Step6: Loop all files

In [14]:
df_log = pd.read_csv("log_categories.csv")
df_old_user_timeline = pd.read_csv("0813_test_multi.csv")

print(df_old_user_timeline.head(2))

# Set the directory path to your CSV files
os.chdir("./data")
files = os.listdir()[1:]

if '.DS_Store' in files:
    files.remove('.DS_Store')
    
print(files)

   usrID  session                                     event category  index  \
0    187        1  landing first time in any Vistorian page  logging      2   
1    187        1                   Support Form Completion  logging      2   

  view  start  end  
0  NaN    0.0  0.0  
1  NaN    0.0  0.0  
['ss_Explorer_2461.csv', 'demo_user_187.csv', 'ss_Explorer_2699.csv', 'ms_explorer_2596.csv', 'data_struggler_626.csv', 'ms_Explorer_user_482.csv', 'ss_Explorer_2094.csv', 'data_strugglers_2736.csv', 'demo_2039.csv', 'data_struggler_519.csv', 'ms_explorer_411.csv']


In [15]:
# current_directory = os.getcwd()
# print(current_directory)
# print(os.system('ls -lr'))

In [16]:
for file in files:
    df = pd.read_csv(file)

    # 1. Get page view
    if 'uuid' in df.columns:
        df['view'] = df.apply(assign_view, axis=1)
        df.drop(['action', 'label', 'uuid'], axis=1, inplace=True)
    else:
        clean_ref('referer', df)
        df.drop(['action', 'label', 'ip'], axis=1, inplace=True)

    # 2. Match logID with title and category
    df = join_log_info(df)
    df = update_log_info(df)
    df = get_category_index(df)

    # 3. Decluster file with unuseful logs
    df = decluster_rows(df)

    # 4. Calculate relative time and combine consecutive logs
    df = relative_time(df)
    df = combine_logs(df)


    # 5. Refine some log titles
    df = refine_logs(df)
    df = fill_vis_view(df)

    # 6. Get session IDs and reset relative time
    df = assign_session(df)
    df = reset_time(df)
    
    # 7. Rename the df to match the Visualization specs
    df = df.rename(columns={'user': 'usrID'})
    df = df.rename(columns={'log_title': 'event'})
    df = df.rename(columns={'log_category': 'category'})
    df = df.rename(columns={'log_category_index': 'index'})
    

    folder_path = "../export"
    full_path = f"{folder_path}/{file}"
    
    df.to_csv(full_path, index=False)

In [17]:
import os
import shutil
import pandas as pd

base_path = "/Users/halcyon/Desktop/vistorian dashboard/"

print(os.system("pwd"))

input_directory = os.path.join(base_path, "user_timeline/export")
export_directory = os.path.join(base_path, "data/combined-logs")

sub_folders = []

for file in os.listdir(input_directory):
    if not file.endswith('.csv'):
        continue

    first_part = file.split('_')[0]

    sub_folder = os.path.join(export_directory, first_part)
    if not os.path.exists(sub_folder):
        os.makedirs(sub_folder)

    shutil.move(os.path.join(input_directory, file), os.path.join(sub_folder, file))

    if sub_folder not in sub_folders:
        sub_folders.append(sub_folder)

for sub_folder in sub_folders:
    csv_files = [f for f in os.listdir(sub_folder) if f.endswith('.csv')]

    combined_csv = pd.concat([pd.read_csv(os.path.join(sub_folder, csv_file)) for csv_file in csv_files], ignore_index=True)

#     session_values = combined_csv['user-session'].tolist()
    combined_csv['userID+session'] = combined_csv.apply(lambda row: str(row['usrID']) + '_' + str(row['userSession']), axis=1)
    session_values_with_userID = combined_csv['userID+session'].tolist()
#     session_values = (combined_csv['usrID'] + "_" + combined_csv['user-session']).tolist()
    print
    new_column_values = []

    counter = 1
    previous_session = session_values_with_userID[0]

    for session in session_values_with_userID:
        if session != previous_session:
            counter += 1
        new_column_values.append(counter)
        previous_session = session

    combined_csv['session'] = new_column_values

    combined_csv.to_csv(os.path.join(sub_folder, 'users-combined-update.csv'), index=False)


/Users/halcyon/Desktop/vistorian dashboard/user_timeline/data
0
